In [20]:
import cost_model
import os
import json
import sys

from pprint import pprint
from IPython.display import SVG, display, HTML

def show_image(url1, url2):
    display(HTML("""<table><tr><td><img style='width:100px' src='{}'/></td>
                               <td><img style='width:100px' src='{}'/></td></tr></table>""".format(url1, url2)))
    
def load_by_designer_tool(export_dir):
    """the directory containing exported designs, 
        there is a group for each designer and each tool"""
    
    all_designs = {}
    for dirname in os.listdir(export_dir):
        design_dir = os.path.join(export_dir, dirname)
        if not os.path.isdir(design_dir):
            continue
            
        # load paths for saved images, relates to saved designs by their canvas ids
        img_dir = os.path.join(design_dir, "images")
        img_file_names = {}
        for fname in os.listdir(img_dir):
            if fname.endswith(".png"):
                if "Alternative 1" in fname:
                    design_id = "Alternative 1"
                elif "Alternative 2" in fname:
                    design_id = "Alternative 2"
                elif "Alternative 3" in fname:
                    design_id = "Alternative 3"
                elif "Original" in fname:
                    design_id = "Original"
                img_file_names[design_id] = os.path.join(img_dir, fname)
                
        group_id = dirname
        designs = []
        with open(os.path.join(design_dir, "saved.json")) as f:
            for d in json.load(f)["saved"]:
                d["img_path"] = img_file_names[d["elements"]["name"]]
                designs.append(d)
        
        if group_id not in all_designs:
            all_designs[group_id] = []
        all_designs[group_id] += designs

    return all_designs


def load_by_tool(export_dir):
    """the directory containing exported designs, 
        there is a group for each tool"""
    
    all_designs = {}
    for dirname in os.listdir(export_dir):
        design_dir = os.path.join(export_dir, dirname)
        if not os.path.isdir(design_dir):
            continue
        
        # load paths for saved images, relates to saved designs by their canvas ids
        img_dir = os.path.join(design_dir, "images")
        img_file_names = {}
        for fname in os.listdir(img_dir):
            if fname.endswith(".png"):
                if "Alternative 1" in fname:
                    design_id = "Alternative 1"
                elif "Alternative 2" in fname:
                    design_id = "Alternative 2"
                elif "Alternative 3" in fname:
                    design_id = "Alternative 3"
                elif "Original" in fname:
                    design_id = "Original"
                img_file_names[design_id] = os.path.join(img_dir, fname)
                
        group_id = "baseline" if "baseline" in dirname else "scout"
        designs = []
        with open(os.path.join(design_dir, "saved.json")) as f:
            for d in json.load(f)["saved"]:
                d["img_path"] = img_file_names[d["elements"]["name"]]
                designs.append(d)
        
        if group_id not in all_designs:
            all_designs[group_id] = []
        all_designs[group_id] += designs

    return all_designs


def analyze_design(all_designs):
    """ Analyze design scores from all saved apps, the scores are normalized across all designs
        For each pair of saved designs in each file, display their diversity score as well as svgs
        Inputs:
            all_designs: a list of saved designs (saved.json)
    """
    
    raw_diversity_score = {}
    for group_id, designs in all_designs.items():

        for i in range(len(designs)):
            for j in range(i + 1, len(designs)):
                di, dj = designs[i], designs[j]
                #diversity_score, diff = cost_model.compute_diversity_score(di["elements"], dj["elements"])
                raw_diversity_score[(group_id, i, j)] = cost_model.compute_unnormalized_diversity_score(
                    di["elements"], dj["elements"])

    final_scores = cost_model.normalize_diversity_scores(raw_diversity_score)
    
    design_pairs = []
    for pair_id in final_scores:
        group_id, i,j = pair_id
        score = (final_scores[pair_id]["alt_group_score"]
                    + final_scores[pair_id]["pos_diff_score"] 
                    + final_scores[pair_id]["rel_dist_diff_score"] 
                    + 2 * final_scores[pair_id]["size_diff_score"])
        design_pair = {
            "group_id": group_id,
            "score": score,
            "score_detail": final_scores[pair_id],
            "di": all_designs[group_id][i],
            "dj": all_designs[group_id][j],
        }
        
        design_pairs.append(design_pair)

    design_pairs.sort(key=lambda x: x['score'], reverse=True)
    for pair in design_pairs: 
        no_svg_to_display = False
        group_id = pair["group_id"]
        di = pair['di']
        dj = pair['dj']
     
        score = pair['score']
        score_detail = pair['score_detail']
        print("# experiments from {}".format(pair["group_id"]))
        print("# diversity score {}".format(score))
        print("# score details")
        pprint(score_detail)

        img_i = di["img_path"]
        img_j = dj["img_path"]
        show_image(img_i, img_j)
        print("")
    
# this is the diversity score for each tool (scout / baseline)
all_designs = load_by_tool("./saved_designs/social_media")
analyze_design(all_designs)

# # this is the diversity score for each designer-tool (each experiment)
# all_designs = load_by_designer_tool("./saved_designs/social_media")
# analyze_design(all_designs)

# experiments from scout
# diversity score 1.4653908893878405
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.5220445814395258,
 'rel_dist_diff_score': 0.38634496869604706,
 'size_diff_score': 0.27850066962613373}


,



# experiments from scout
# diversity score 1.4651730690233116
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.49346455255892835,
 'rel_dist_diff_score': 0.35910190406179826,
 'size_diff_score': 0.30630330620129254}


,



# experiments from baseline
# diversity score 1.4482961311754539
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.6874265257500035,
 'rel_dist_diff_score': 0.4471243234998517,
 'size_diff_score': 0.15687264096279932}


...

,



# experiments from scout
# diversity score 0.0
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.0,
 'rel_dist_diff_score': 0.0,
 'size_diff_score': 0.0}


,



# experiments from scout
# diversity score 0.0
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.0,
 'rel_dist_diff_score': 0.0,
 'size_diff_score': 0.0}


,
